## DATA ANALYSIS: LEDGER

### Import dependancies

In [6]:
import duckdb
import json
import pandas as pd
import matplotlib.pyplot
import numpy

### Data source

In [24]:
data_source_ledger_yr_1 = "../data/parquet/financial_journal_2022.parquet"
data_source_ledger_yr_2 = "../data/parquet/financial_journal_2023.parquet"

data_souce_account_chart = "../data/accounts_chart.json"


### Load & merge ledger data

In [27]:
conn = duckdb.connect(database=':memory:') 

conn.execute(f"""
    CREATE TABLE financial_reports AS
    SELECT *
    FROM read_parquet('{data_source_ledger_yr_1}')
""")

df_yr1 = conn.execute("SELECT * FROM financial_reports").fetchdf()

In [ ]:
conn = duckdb.connect(database=':memory:') 

conn.execute(f"""
    CREATE TABLE financial_reports AS
    SELECT *
    FROM read_parquet('{data_source_ledger_yr_2}')
""")

df_yr2 = conn.execute("SELECT * FROM financial_reports").fetchdf()

df = pd.concat([df_yr1, df_yr2])

### Load account chart into df

In [30]:
with open(data_souce_account_chart, "r", encoding="utf-8") as file:
    data_source_account_chart = json.load(file)

df_accounts_chart = pd.DataFrame(list(data_source_account_chart.items()), columns=['account', 'description'])
